#### Notebook to merge datasets from multiple Excel files, remove the duplicates and incorporate location data. Customer data is anonymized using the customer information Excel file.

In [ ]:
import pandas as pd
import numpy as np

Note: the files DiagUsagePowerBI_jan01.xlsx and DiagUsagePowerBI_feb09.xlsx spreadsheets are not included in the Capstone repo because they contain ALLDATA customer information

In [84]:
#merge excel files from multiple dates and remove the duplicates
df1 = pd.read_excel('../Data/DiagUsagePowerBI_jan01.xlsx', sheet_name='connect_system')
df2 = pd.read_excel('../Data/DiagUsagePowerBI_feb09.xlsx', sheet_name='connect_system')

In [85]:
diag_usage_df = pd.concat([df1, df2])

In [86]:
num_duplicates = diag_usage_df.duplicated().sum()
print(f'Number of duplicates: {num_duplicates}')

Number of duplicates: 79823


In [90]:
diag_usage_df = diag_usage_df.drop_duplicates()
diag_usage_df['siteId'] = diag_usage_df['siteId'].fillna(0).astype(int)

In [91]:
#remove rows with siteId = 0
diag_usage_df = diag_usage_df[diag_usage_df['siteId'] != 0]

In [92]:
diag_usage_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 190263 entries, 0 to 133650
Data columns (total 17 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   timestamp        190263 non-null  datetime64[ns]
 1   siteId           190263 non-null  int32         
 2   shop_info        190263 non-null  object        
 3   billable_status  185746 non-null  object        
 4   OsName           190263 non-null  object        
 5   client_OS        190263 non-null  object        
 6   vin              181137 non-null  object        
 7   year             180572 non-null  float64       
 8   make             180569 non-null  object        
 9   model            180572 non-null  object        
 10  system           190262 non-null  object        
 11  veh_desc         190263 non-null  object        
 12  vci_version      189349 non-null  object        
 13  scan_method      190263 non-null  object        
 14  batt_voltage     139506 n

In [93]:
#read the customer data
df3 = pd.read_excel('../Data/Customer_Data.xlsx', sheet_name='diag_site_w_prod_type')

In [94]:
mapping_dict = dict(zip(df3['SITE_ID'], df3['SITE_ID_ANON']))

In [ ]:
#anonymize the siteId from customer data
diag_usage_df['siteId'] = diag_usage_df['siteId'].map(mapping_dict)

In [98]:
diag_usage_df.sample(5)

,timestamp,siteId,shop_info,billable_status,OsName,client_OS,vin,year,make,model,system,veh_desc,vci_version,scan_method,batt_voltage,result_string,result
129299,2024-10-11 00:00:09.166,706759,DESTINATION MAZDA 195230,BILL,Android,Android 14,NaN,NaN,NaN,NaN,Dynamic Stability Control (DSC_10),Dynamic Stability Control (DSC_10),21.3.2337.1,manual scan,NaN,Successfully connected to vehicle,success
124128,2024-10-10 17:13:33.097,496026,PLACERVILLE BODY SHOP 178100,BILL,Android,Android 14,1GT424E87FF,2015.0,Sierra,Sierra,Enhanced Powertrain,2015 GMC Sierra Enhanced Powertrain,22.1.2418.1,manual scan,NaN,Successfully connected to vehicle,success
27969,2025-01-21 18:26:34.210,52399,CIRCUIT COLLISION 159099,BILL,Android,Android 10,2HGFB2F50DH,2013.0,Honda,Civic,Anti-Lock Brakes,2013 Honda Civic Anti-Lock Brakes,22.3.2439.2,manual scan,10.55,<div>Communications could not be established w...,failed
16826,2024-12-18 14:30:41.984,200673,Carrosserie CertiPlus 9199,BILL,Android,Android 8.1.0,2HKRW2H44LH,2020.0,Honda,CR-V,Enhanced Powertrain,2020 Honda CR-V Enhanced Powertrain,22.3.2439.2,quick_scan,10.87,Successfully connected to vehicle,success
32045,2025-01-22 21:47:46.458,503311,NORTHEAST COLLISION - ELMA 77786,BILL,Android,Android 14,5J6RE4H72AL,2010.0,Honda,CR-V,Enhanced Powertrain,2010 Honda CR-V Enhanced Powertrain,22.3.2439.2,quick_scan,11.69,<div>Communications could not be established w...,failed


In [99]:
#copy city, state, zip code and customer type from customer data
diag_usage_df['state'] = diag_usage_df['siteId'].map(dict(zip(df3['SITE_ID_ANON'], df3['STATE'])))
diag_usage_df['city'] = diag_usage_df['siteId'].map(dict(zip(df3['SITE_ID_ANON'], df3['CITY'])))
diag_usage_df['product_type'] = diag_usage_df['siteId'].map(dict(zip(df3['SITE_ID_ANON'], df3['MODULE_NAME'])))
diag_usage_df['postal_code'] = diag_usage_df['siteId'].map(dict(zip(df3['SITE_ID_ANON'], df3['POSTAL_CODE'])))

In [101]:
diag_usage_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 190263 entries, 0 to 133650
Data columns (total 21 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   timestamp        190263 non-null  datetime64[ns]
 1   siteId           190263 non-null  int64         
 2   shop_info        190263 non-null  object        
 3   billable_status  185746 non-null  object        
 4   OsName           190263 non-null  object        
 5   client_OS        190263 non-null  object        
 6   vin              181137 non-null  object        
 7   year             180572 non-null  float64       
 8   make             180569 non-null  object        
 9   model            180572 non-null  object        
 10  system           190262 non-null  object        
 11  veh_desc         190263 non-null  object        
 12  vci_version      189349 non-null  object        
 13  scan_method      190263 non-null  object        
 14  batt_voltage     139506 n

In [102]:
#replace missing values in city, state, product_type and postal_code with 'Unknown'
diag_usage_df['city'] = diag_usage_df['city'].fillna('unknown')
diag_usage_df['state'] = diag_usage_df['state'].fillna('unknown')
diag_usage_df['product_type'] = diag_usage_df['product_type'].fillna('unknown')
diag_usage_df['postal_code'] = diag_usage_df['postal_code'].fillna('unknown')

In [103]:
diag_usage_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 190263 entries, 0 to 133650
Data columns (total 21 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   timestamp        190263 non-null  datetime64[ns]
 1   siteId           190263 non-null  int64         
 2   shop_info        190263 non-null  object        
 3   billable_status  185746 non-null  object        
 4   OsName           190263 non-null  object        
 5   client_OS        190263 non-null  object        
 6   vin              181137 non-null  object        
 7   year             180572 non-null  float64       
 8   make             180569 non-null  object        
 9   model            180572 non-null  object        
 10  system           190262 non-null  object        
 11  veh_desc         190263 non-null  object        
 12  vci_version      189349 non-null  object        
 13  scan_method      190263 non-null  object        
 14  batt_voltage     139506 n

In [104]:
#save a version with shop info which contains the original siteId
diag_usage_df.to_excel('DiagUsagePowerBI_merged_shop_info.xlsx', index=False)

In [105]:
#drop the shop_info column to anonymize the data
diag_usage_df = diag_usage_df.drop(columns=['shop_info'])

In [ ]:
#save completly anonymized and merged data
diag_usage_df.to_excel('DiagUsagePowerBI_merged.xlsx', index=False)